<a href="https://colab.research.google.com/github/rootloginson/X-Ray-Image-Covid19-Detection-Project/blob/master/Final/make_CSV_for_test2and3_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
## It'll look better and intuitive on Google Colab Environment (◕‿◕✿)

---

In [ ]:
# This part probably will be enough for utilizing Drive in Colab
# but examine links above if you encounter with problems.
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



# *Import Libraries and path

In [ ]:
# All libraries are already presented in Colab Servers, we don't need to install anything with pip
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import models
import os 

In [ ]:
wd = '/content/drive/MyDrive/Colab Notebooks/Final'

# Creating DataFrame for CustomDataset

### Reading .csv, 
- *Class*(*Labels*) column added (nf: dataframe name )
- Image name setted as index name
- Image dimension list created (gray, rgb, rgba.)
- label_dict (object), for class(label) explanations


### Image channel dimension list 
list of (gray, rgb, rgba etc.)


In [ ]:
from torchvision.datasets import ImageFolder

In [ ]:
from os import listdir
from os.path import isfile, join

mypath0 = wd + '/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/0'
mypath3 = wd + '/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/3'

dir_path = [mypath0, mypath3]
name_class = [[f,int(d[-1])] for d in dir_path for f in listdir(d)]

In [ ]:
ekdata = pd.DataFrame(name_class, columns=['X_ray_image_name', 'Class'])

In [ ]:
ekdata.head()

,X_ray_image_name,Class
0,IM-0220-0001.jpeg,0
1,IM-0214-0001.jpeg,0
2,IM-0264-0001.jpeg,0
3,IM-0217-0001.jpeg,0
4,IM-0216-0001.jpeg,0


In [ ]:
ekdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   X_ray_image_name  98 non-null     object
 1   Class             98 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [ ]:
ekdata.Class.value_counts()

3    70
0    28
Name: Class, dtype: int64

In [ ]:
# create copy of nf dataframe to work with
#chdim_df = nf.copy()
test_normal = ekdata[ekdata['Class']==0]
test_covid = ekdata[ekdata['Class']==3]

In [ ]:
train_chdim = test_normal.copy()
test_chdim = test_covid.copy()

In [ ]:
channel_dimensions_dict = {}

# for train file
for i in range(len(train_chdim.index)):
  # corresponding image name
  img_name = train_chdim.iloc[i]['X_ray_image_name'] 
  # open image
  img = Image.open(wd + '/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/0'+ '/' + img_name)
  # number of channel an image has
  dim = img.getbands()

  if dim not in channel_dimensions_dict:
    channel_dimensions_dict[dim] = []

  # add image name and channel dim to a list. 
  channel_dimensions_dict[dim].append(img_name)
  # I deleted the set after using dictionary

# for test file
for i in range(len(test_chdim.index)):
  # corresponding image name
  img_name = test_chdim.iloc[i]['X_ray_image_name'] 
  # open image
  img = Image.open(wd + '/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/3'+ '/' + img_name)
  # number of channel an image has
  dim = img.getbands()

  if dim not in channel_dimensions_dict:
    channel_dimensions_dict[dim] = []

  # add image name and channel dim to a list. 
  channel_dimensions_dict[dim].append(img_name)
  # I deleted the set after using dictionary


# {dim:[img_name, ...], dim:[img_name] ...}
print("There are >>", channel_dimensions_dict.keys() , "<< channel Images\n")

# img quantity in channels
dict_info = [(k, len(v)) for k, v in channel_dimensions_dict.items()]
print("image quantity of channels")
print(dict_info)
print("\ntotal")
print(sum(i[1] for i in dict_info))

There are >> dict_keys([('L',), ('R', 'G', 'B'), ('R', 'G', 'B', 'A')]) << channel Images

image quantity of channels
[(('L',), 32), (('R', 'G', 'B'), 55), (('R', 'G', 'B', 'A'), 11)]

total
98


In [ ]:
# create a dict of{img_name:img_dim}
reversed_channel_dict = {v_in:len(k) for (k,v) in channel_dimensions_dict.items() for v_in in v}
# create data set of channel dimensions. row: ImageName, column: Channel_dim
dataset_channel_dim_df = pd.DataFrame.from_dict(reversed_channel_dict, orient='index', columns=['Channel_dim'])

print(dataset_channel_dim_df.value_counts())
print("\n size:",dataset_channel_dim_df.size)

Channel_dim
3              55
1              32
4              11
dtype: int64

 size: 98


In [ ]:
save_path_of_channel_dim = wd+'/KaggleCovid/Updated_Data_Frames/dataset_channel_dim_of_test3.csv'
dataset_channel_dim_df.to_csv(save_path_of_channel_dim, index_label='X_ray_image_name')

### Adding channel dim

In [ ]:
# sorted channel dim, index is image name
channel_dim_list = pd.read_csv(wd+'/KaggleCovid/Updated_Data_Frames/dataset_channel_dim_of_test3.csv', index_col=0).sort_index()

In [ ]:
# kaggle_dataset, index=X_ray_image_name, column=Class
nf_temp = ekdata[['X_ray_image_name', 'Class']].set_index('X_ray_image_name').sort_index()

In [ ]:
channel_dim_list.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98 entries, IM-0214-0001.jpeg to covid (9).png
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Channel_dim  98 non-null     int64
dtypes: int64(1)
memory usage: 1.5+ KB


In [ ]:
nf_temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98 entries, IM-0214-0001.jpeg to covid (9).png
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Class   98 non-null     int64
dtypes: int64(1)
memory usage: 1.5+ KB


---
*Adding Channel dims to X_ray_image_name, Class(Label) dataset*


In [ ]:
nf_temp['Channel_dim'] = channel_dim_list['Channel_dim']

# Save Processed csv. 
save_path_of_processed_dataset = wd+'/KaggleCovid/Updated_Data_Frames/processed_dataset_test3.csv'
nf_temp.to_csv(save_path_of_processed_dataset, index_label='X_ray_image_name')

# *Processed Dataframe (--check point)
for processsed dataframe call ***get_processedDataframe()*** function

for kaggle dataframe ***get_kaggleDataframe()*** function


In [ ]:
def get_test3Dataframe():
  #already sorted
  checkpoint = pd.read_csv(wd+'/KaggleCovid/Updated_Data_Frames/processed_dataset_test3.csv', index_col=0)
  return checkpoint


In [ ]:
test3 = get_test3Dataframe()
test3.head()

,Class,Channel_dim
X_ray_image_name,,
IM-0214-0001.jpeg,0,1
IM-0215-0001.jpeg,0,1
IM-0216-0001.jpeg,0,1
IM-0217-0001.jpeg,0,1
IM-0218-0001.jpeg,0,1


In [ ]:
# check if there is a null "Label"
test3['Channel_dim'].isnull().sum()

0

In [ ]:
test3.Channel_dim.value_counts()

3    55
1    32
4    11
Name: Channel_dim, dtype: int64

In [ ]:
len(test3.index)

98

---
Covid19 Images 

><font color='gray'>**GRAY**</font>  = **32**

><font color='PaleVioletRed'>**R**</font><font color='90EE90'>**G**</font><font color='00BFFF'>**B**</font> = **55**

><font color='PaleVioletRed'>**R**</font><font color='90EE90'>**G**</font><font color='00BFFF'>**B**</font>
<font color='Black'>- A</font> = **11**

In [ ]:
test3.loc[test3['Class']==3].value_counts('Channel_dim')

Channel_dim
3    55
4    11
1     4
dtype: int64

# *Move images to directory (--all in one function)



In [ ]:
wdt

'/content/drive/MyDrive/Colab Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test2'

In [ ]:
!mv  -v /content/drive/MyDrive/Colab\ Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/3/* /content/drive/MyDrive/Colab\ Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3 

renamed '/content/drive/MyDrive/Colab Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/3/covid (10).jpeg' -> '/content/drive/MyDrive/Colab Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/covid (10).jpeg'
renamed '/content/drive/MyDrive/Colab Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/3/covid (10).jpg' -> '/content/drive/MyDrive/Colab Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/covid (10).jpg'
renamed '/content/drive/MyDrive/Colab Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/3/covid (10).png' -> '/content/drive/MyDrive/Colab Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/covid (10).png'
renamed '/content/drive/MyDrive/Colab Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/3/covid (11).jpeg' -> '/content/drive/MyDrive/Colab Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/covid (11).jpeg'
renamed '/content/drive/MyDrive/Colab Notebooks/Final/KaggleCovi

In [ ]:
!mv  -v /content/drive/MyDrive/Colab\ Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/0/* /content/drive/MyDrive/Colab\ Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3

renamed '/content/drive/MyDrive/Colab Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/0/IM-0214-0001.jpeg' -> '/content/drive/MyDrive/Colab Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/IM-0214-0001.jpeg'
renamed '/content/drive/MyDrive/Colab Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/0/IM-0215-0001.jpeg' -> '/content/drive/MyDrive/Colab Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/IM-0215-0001.jpeg'
renamed '/content/drive/MyDrive/Colab Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/0/IM-0216-0001.jpeg' -> '/content/drive/MyDrive/Colab Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/IM-0216-0001.jpeg'
renamed '/content/drive/MyDrive/Colab Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/0/IM-0217-0001.jpeg' -> '/content/drive/MyDrive/Colab Notebooks/Final/KaggleCovid/Coronahack-Chest-XRay-Dataset/test3/IM-0217-0001.jpeg'
renamed '/content/drive/MyDrive/Colab Notebo